In [ ]:
from sklearn.linear_model import Lasso
import numpy as np
import pandas as pd
import os
#------DATA PREPARATION---------#
# Cargar datos
file = "C:/Users/GA.HornaM/Documents/GitHub/CausalAI-Course/data/wage2015_subsample_inference.csv"
q3 = pd.read_csv(file)

# Obtener el directorio de trabajo actual
directorio_actual = os.getcwd()
print("Directorio actual:", directorio_actual)
n, nvar = q3.shape[0], q3.shape[1]

# Crear la variable 'id'
q3['id'] = np.arange(1, n+1, 1)

# ---- Split ---- #
np.random.seed(1234)
random_indices = np.random.choice(n, size=int(n*0.8), replace=False)  # 80/20 split

# Training set
train = q3.iloc[random_indices]
index_train = train['id'].tolist()  # Índices originales para el conjunto de entrenamiento
ntrain = train.shape[0]

# Testing set
test = q3.drop(random_indices)
index_test = test['id'].tolist()  # Índices originales para el conjunto de prueba
ntest = test.shape[0]

print(np.intersect1d(index_train, index_test))  # --> 0
#---(2) alpha range------#
#-------------------------#
alphas = np.arange(0.1, 0.6, 0.1)
# Definir el número de folds
k = 5

# Calcular el número de observaciones por fold
obsfold = ntrain // k  # División entera para obtener el número entero de observaciones por fold

# Calcular los índices de corte
cutoff = [obsfold * i for i in range(1, k+1)]

# Definir los índices para cada fold
index_f1 = index_train[0:cutoff[0]]
index_f2 = index_train[cutoff[0]:cutoff[1]]
index_f3 = index_train[cutoff[1]:cutoff[2]]
index_f4 = index_train[cutoff[2]:cutoff[3]]
index_f5 = index_train[cutoff[3]:cutoff[4]]


from sklearn.linear_model import LassoCV
from sklearn.model_selection import KFold


# Definir las variables predictoras 'X' y la variable objetivo 'y'
X = q3.drop(columns=['wage', 'lwage', 'id'])
y = q3['wage']

# Definir los valores de alpha 
alphas = np.arange(0.1, 0.6, 0.1)

np.random.seed(1234)

# Definir el número de folds para la validación cruzada
k = 5
kf = KFold(n_splits=k)
# Lista para almacenar los MSE de cada alpha en cada fold
MSE_list = []

for alpha in alphas:
    # Inicializar el modelo LassoCV con el alpha actual y KFold
    lasso = LassoCV(alphas=[alpha], cv=kf)
    
    # Realizar la validación cruzada
    MSE_folds = []
    for train_index, val_index in kf.split(X):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]
        
        lasso.fit(X_train, y_train)
        y_pred = lasso.predict(X_val)
        
        # Calcular el MSE para el fold actual
        mse = np.mean((y_val - y_pred)**2)
        MSE_folds.append(mse)
    
    # Calcular el promedio de los MSE de los folds
    avg_mse = np.mean(MSE_folds)
    
    # Agregar el MSE promedio a la lista
    MSE_list.append(avg_mse)

# Imprimir los MSE para cada alpha
for alpha, mse in zip(alphas, MSE_list):
    print(f"Alpha: {alpha}, MSE: {mse}")
#Results 
#Alpha: 0.1, MSE: 401.58570325958806
#Alpha: 0.2, MSE: 400.1433655945446
#Alpha: 0.30000000000000004, MSE: 401.18011478789003
#Alpha: 0.4, MSE: 402.80545535028784
#Alpha: 0.5, MSE: 405.2573945568647